# 🎯 악성 판매자 탐지 결과 분석 (인터랙티브)

**Plotly를 사용한 인터랙티브 분석**
- 마우스를 올리면 상세 정보 표시
- 클릭 드래그로 확대/축소
- 범례 클릭으로 데이터 숨기기/보이기

## 📦 패키지 Import

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ 패키지 import 완료")

✅ 패키지 import 완료


## 🎨 Feature 이름 설정

In [2]:
# Feature 한글 이름
FEATURE_NAMES = {
    'review_density': '리뷰 밀도',
    'question_density': '문의 밀도', 
    'avg_rating': '평균 평점',
    'negative_sentiment_ratio': '부정 감성 비율',
    'rating_sentiment_gap': '평점-감성 괴리도',
    'question_review_ratio': '문의/리뷰 비율'
}

HEATMAP_NAMES = {
    'review_density': '리뷰 밀도',
    'question_density': '문의 밀도',
    'avg_rating': '평균 평점',
    'rating_std': '평점 표준편차',
    'negative_sentiment_ratio': '부정 감성 비율',
    'rating_sentiment_gap': '평점-감성 괴리도',
    'question_review_ratio': '문의/리뷰 비율',
    'abusive_label': '악성 라벨'
}

LABEL_NAMES = {0: '정상', 1: '악성'}

print("✅ Feature 이름 설정 완료")

✅ Feature 이름 설정 완료


## 📊 1. 데이터 로드

In [3]:
print("=" * 70)
print("데이터 로드 중...")
print("=" * 70)

df = pd.read_csv('../output/seller_features.csv')
df['label_name'] = df['abusive_label'].map(LABEL_NAMES)

print(f"\n✅ 데이터 크기: {df.shape}")
print(f"   - 판매자: {len(df)}명")
print(f"   - Feature: {len(df.columns)}개")

df.head()

데이터 로드 중...

✅ 데이터 크기: (374, 24)
   - 판매자: 374명
   - Feature: 24개


,vendor_name,avg_rating,rating_std,low_rating_ratio,avg_review_length,duplicate_review_ratio,negative_keyword_ratio,textless_5star_ratio,review_count,review_density,...,question_density,question_review_ratio,negative_sentiment_ratio,avg_sentiment_score,rating_normalized,rating_sentiment_gap,product_count,abusive_label,conditions_met_count,label_name
0,(주)경일종합식품,4.950000,0.223607,0.000000,468.700000,0.000000,0.050000,0.421053,20,20.0,...,19.0,0.950000,0.050000,0.950000,0.987500,0.037500,1,1,4,악성
1,(주)그린바드,4.750000,0.910465,0.050000,66.400000,0.125000,0.000000,0.611111,20,20.0,...,12.0,0.600000,0.000000,1.000000,0.937500,0.062500,1,0,1,정상
2,(주)다노트정보,5.000000,0.000000,0.000000,414.421053,0.000000,0.157895,0.473684,19,19.0,...,17.0,0.894737,0.157895,0.842105,1.000000,0.157895,1,1,5,악성
3,(주)리빙스토리,4.333333,1.278019,0.142857,145.571429,0.142857,0.047619,0.500000,21,21.0,...,2.0,0.095238,0.047619,0.952381,0.833333,0.119048,1,0,0,정상
4,(주)맥스올(MAXALL),4.700000,0.732695,0.050000,283.400000,0.083333,0.100000,0.312500,20,20.0,...,16.0,0.800000,0.100000,0.900000,0.925000,0.025000,1,0,2,정상


## 📈 2. 라벨 분포 분석 (인터랙티브)

In [4]:
print("=" * 70)
print("라벨 분포 분석")
print("=" * 70)

label_counts = df['label_name'].value_counts()

# 통계 출력
for label, count in label_counts.items():
    percentage = count / len(df) * 100
    print(f"{label} 판매자: {count}명 ({percentage:.1f}%)")

# 인터랙티브 파이 차트
fig = go.Figure(data=[go.Pie(
    labels=label_counts.index,
    values=label_counts.values,
    hole=0.3,
    marker=dict(colors=['#2ecc71', '#e74c3c']),
    textinfo='label+percent+value',
    hovertemplate='<b>%{label}</b><br>' +
                  '판매자 수: %{value}명<br>' +
                  '비율: %{percent}<br>' +
                  '<extra></extra>'
)])

fig.update_layout(
    title='판매자 라벨 분포 (마우스를 올려보세요!)',
    height=500,
    showlegend=True
)

fig.show()

# 막대 차트
fig2 = go.Figure(data=[go.Bar(
    x=label_counts.index,
    y=label_counts.values,
    marker=dict(color=['#2ecc71', '#e74c3c']),
    text=label_counts.values,
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>판매자 수: %{y}명<extra></extra>'
)])

fig2.update_layout(
    title='판매자 라벨 분포',
    xaxis_title='판매자 유형',
    yaxis_title='판매자 수',
    height=500
)

fig2.show()

라벨 분포 분석
정상 판매자: 282명 (75.4%)
악성 판매자: 92명 (24.6%)


## 🎪 3. 조건 충족 개수 분석 (인터랙티브)

In [5]:
print("=" * 70)
print("조건 충족 개수 분석")
print("=" * 70)

conditions_dist = df['conditions_met_count'].value_counts().sort_index()

# 통계 출력
print("\n조건별 분포:")
for count, freq in conditions_dist.items():
    status = LABEL_NAMES[1] if count >= 3 else LABEL_NAMES[0]
    print(f"  {count}개 충족: {freq}명 ({status})")

# 인터랙티브 막대 차트
colors_map = {0: '#27ae60', 1: '#2ecc71', 2: '#3498db', 
              3: '#f39c12', 4: '#e67e22', 5: '#e74c3c', 6: '#c0392b'}
bar_colors = [colors_map.get(i, '#95a5a6') for i in conditions_dist.index]

fig = go.Figure()

# 막대 그래프
fig.add_trace(go.Bar(
    x=conditions_dist.index,
    y=conditions_dist.values,
    marker=dict(color=bar_colors),
    text=conditions_dist.values,
    texttemplate='%{text}명',
    textposition='outside',
    hovertemplate='조건 %{x}개 충족<br>판매자 수: %{y}명<extra></extra>'
))

# 악성 기준선
fig.add_vline(x=2.5, line_dash="dash", line_color="red", 
              annotation_text="악성 기준선 (3개 이상)", 
              annotation_position="top")

fig.update_layout(
    title='악성 판매자 조건 충족 개수 분포',
    xaxis_title='충족한 조건 개수',
    yaxis_title='판매자 수',
    height=600,
    showlegend=False
)

fig.show()

조건 충족 개수 분석

조건별 분포:
  0개 충족: 48명 (정상)
  1개 충족: 136명 (정상)
  2개 충족: 98명 (정상)
  3개 충족: 61명 (악성)
  4개 충족: 22명 (악성)
  5개 충족: 9명 (악성)


## 🎨 4. Feature 비교 (Box Plot - 인터랙티브)

In [6]:
print("=" * 70)
print("정상 vs 악성 Feature 비교")
print("=" * 70)

key_features = [
    'review_density', 'question_density', 'avg_rating',
    'negative_sentiment_ratio', 'rating_sentiment_gap', 'question_review_ratio'
]

normal = df[df['label_name'] == '정상']
abusive = df[df['label_name'] == '악성']

# 통계 출력
print("\n평균값 비교:")
comparison = pd.DataFrame({
    'Feature': [FEATURE_NAMES[f] for f in key_features],
    '정상': [normal[f].mean() for f in key_features],
    '악성': [abusive[f].mean() for f in key_features],
    '차이': [abusive[f].mean() - normal[f].mean() for f in key_features]
})
print(comparison.round(3).to_string(index=False))

# Subplot 생성 (2x3)
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[FEATURE_NAMES[f] + ' 비교' for f in key_features],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

for idx, feature in enumerate(key_features):
    row = idx // 3 + 1
    col = idx % 3 + 1
    
    # 정상
    fig.add_trace(
        go.Box(
            y=normal[feature],
            name='정상',
            marker_color='#2ecc71',
            boxmean='sd',
            hovertemplate='<b>정상</b><br>' + 
                          FEATURE_NAMES[feature] + ': %{y:.3f}<br>' +
                          '<extra></extra>',
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )
    
    # 악성
    fig.add_trace(
        go.Box(
            y=abusive[feature],
            name='악성',
            marker_color='#e74c3c',
            boxmean='sd',
            hovertemplate='<b>악성</b><br>' + 
                          FEATURE_NAMES[feature] + ': %{y:.3f}<br>' +
                          '<extra></extra>',
            showlegend=(idx == 0)
        ),
        row=row, col=col
    )

fig.update_layout(
    title_text='정상 vs 악성 Feature 비교 (클릭하여 확대/축소)',
    height=900,
    showlegend=True
)

fig.show()

정상 vs 악성 Feature 비교

평균값 비교:
  Feature     정상     악성     차이
    리뷰 밀도 20.506 19.918 -0.587
    문의 밀도 12.918 15.326  2.408
    평균 평점  4.667  4.790  0.123
 부정 감성 비율  0.069  0.157  0.087
평점-감성 괴리도  0.075  0.142  0.067
 문의/리뷰 비율  0.625  0.805  0.181


## 🔥 5. 상관관계 히트맵 (인터랙티브)

In [7]:
print("=" * 70)
print("Feature 상관관계 분석")
print("=" * 70)

correlation_features = [
    'review_density', 'question_density', 'avg_rating', 'rating_std',
    'negative_sentiment_ratio', 'rating_sentiment_gap', 
    'question_review_ratio', 'abusive_label'
]

corr_matrix = df[correlation_features].corr()

# 한글 이름으로 변경
corr_labels = [HEATMAP_NAMES[f] for f in correlation_features]

# 인터랙티브 히트맵
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_labels,
    y=corr_labels,
    colorscale='RdYlGn_r',
    zmid=0,
    text=corr_matrix.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 10},
    hovertemplate='%{y}<br>%{x}<br>상관계수: %{z:.3f}<extra></extra>'
))

fig.update_layout(
    title='Feature 상관관계 히트맵 (마우스를 올려 상세 정보 확인)',
    height=700,
    xaxis={'side': 'bottom'},
    yaxis={'autorange': 'reversed'}
)

fig.update_xaxes(tickangle=45)

fig.show()

# 라벨과의 상관관계 출력
print("\n악성 라벨과의 상관관계 (절대값 높은 순):")
label_corr = corr_matrix['abusive_label'].drop('abusive_label').abs().sort_values(ascending=False)
for feature, corr_val in label_corr.items():
    actual_corr = corr_matrix.loc[feature, 'abusive_label']
    print(f"  {feature:30s}: {actual_corr:+.3f}")

Feature 상관관계 분석



악성 라벨과의 상관관계 (절대값 높은 순):
  negative_sentiment_ratio      : +0.439
  rating_sentiment_gap          : +0.332
  question_review_ratio         : +0.280
  question_density              : +0.228
  rating_std                    : -0.170
  avg_rating                    : +0.133
  review_density                : -0.090


## 🎯 6. 산점도 - 부정 감성 vs 평점 괴리

In [8]:
# 인터랙티브 산점도
fig = px.scatter(
    df,
    x='negative_sentiment_ratio',
    y='rating_sentiment_gap',
    color='label_name',
    size='question_count',
    hover_data={
        'vendor_name': True,
        'negative_sentiment_ratio': ':.3f',
        'rating_sentiment_gap': ':.3f',
        'avg_rating': ':.2f',
        'question_count': True,
        'conditions_met_count': True,
        'label_name': False
    },
    color_discrete_map={'정상': '#2ecc71', '악성': '#e74c3c'},
    title='부정 감성 비율 vs 평점-감성 괴리도 (점 크기 = 문의 개수)',
    labels={
        'negative_sentiment_ratio': '부정 감성 비율',
        'rating_sentiment_gap': '평점-감성 괴리도',
        'label_name': '판매자 유형',
        'vendor_name': '판매자명',
        'question_count': '문의 개수',
        'conditions_met_count': '조건 충족 개수',
        'avg_rating': '평균 평점'
    },
    height=600
)

fig.update_traces(marker=dict(line=dict(width=1, color='white')))
fig.show()

## 🔍 7. 주요 판매자 분석 (인터랙티브 테이블)

In [9]:
print("=" * 70)
print("주요 판매자 분석")
print("=" * 70)

# 가장 의심스러운 판매자
top_suspicious = df.nlargest(10, 'conditions_met_count')[[
    'vendor_name', 'conditions_met_count', 'label_name',
    'negative_sentiment_ratio', 'rating_sentiment_gap', 'avg_rating'
]].round(3)

print("\n=== 가장 의심스러운 판매자 (Top 10) ===")
print(top_suspicious.to_string(index=False))

# 인터랙티브 테이블
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['판매자명', '조건 충족', '라벨', '부정 감성', '평점 괴리', '평균 평점'],
        fill_color='#e74c3c',
        font=dict(color='white', size=12),
        align='center'
    ),
    cells=dict(
        values=[top_suspicious[col] for col in top_suspicious.columns],
        fill_color=[['#ffe6e6' if i % 2 == 0 else 'white' for i in range(len(top_suspicious))]],
        align='left',
        font=dict(size=11)
    )
)])

fig.update_layout(
    title='가장 의심스러운 판매자 Top 10',
    height=400
)

fig.show()

# 가장 건전한 판매자
top_healthy = df.nsmallest(10, 'conditions_met_count')[[
    'vendor_name', 'conditions_met_count', 'label_name',
    'negative_sentiment_ratio', 'rating_sentiment_gap', 'avg_rating'
]].round(3)

print("\n=== 가장 건전한 판매자 (Top 10) ===")
print(top_healthy.to_string(index=False))

fig2 = go.Figure(data=[go.Table(
    header=dict(
        values=['판매자명', '조건 충족', '라벨', '부정 감성', '평점 괴리', '평균 평점'],
        fill_color='#2ecc71',
        font=dict(color='white', size=12),
        align='center'
    ),
    cells=dict(
        values=[top_healthy[col] for col in top_healthy.columns],
        fill_color=[['#e6ffe6' if i % 2 == 0 else 'white' for i in range(len(top_healthy))]],
        align='left',
        font=dict(size=11)
    )
)])

fig2.update_layout(
    title='가장 건전한 판매자 Top 10',
    height=400
)

fig2.show()

주요 판매자 분석

=== 가장 의심스러운 판매자 (Top 10) ===
vendor_name  conditions_met_count label_name  negative_sentiment_ratio  rating_sentiment_gap  avg_rating
   (주)다노트정보                     5         악성                     0.158                 0.158       5.000
        부스터                     5         악성                     0.200                 0.175       4.900
       에스엔샵                     5         악성                     0.150                 0.150       5.000
        올디얼                     5         악성                     0.200                 0.175       4.900
        올픽스                     5         악성                     0.350                 0.275       4.700
        정앤정                     5         악성                     0.200                 0.188       4.950
티케이씨제트 주식회사                     5         악성                     0.200                 0.200       5.000
       플랜가전                     5         악성                     0.158                 0.092       4.737
         필리   


=== 가장 건전한 판매자 (Top 10) ===
                            vendor_name  conditions_met_count label_name  negative_sentiment_ratio  rating_sentiment_gap  avg_rating
                               (주)리빙스토리                     0         정상                     0.048                 0.119       4.333
(주)어거스트포코리아(AUGUST FOUR KOREA Co.,Ltd.)                     0         정상                     0.048                 0.012       4.762
           (주)케이지에프앤비 (KG F&B Co., ltd)                     0         정상                     0.048                 0.000       4.810
                              (주)큐브온커머스                     0         정상                     0.122                 0.079       4.829
                                  J.M유통                     0         정상                     0.048                 0.024       4.905
                                   골든상사                     0         정상                     0.048                 0.024       4.714
                                     그릿 

## 💾 8. 결과 저장

In [10]:
print("=" * 70)
print("분석 결과 저장")
print("=" * 70)

normal = df[df['label_name'] == '정상']
abusive = df[df['label_name'] == '악성']

if len(normal) > 0 and len(abusive) > 0:
    key_features = [
        'review_density', 'question_density', 'avg_rating',
        'negative_sentiment_ratio', 'rating_sentiment_gap', 'question_review_ratio'
    ]
    
    summary = pd.DataFrame({
        'Feature': key_features,
        '정상_평균': [normal[f].mean() for f in key_features],
        '악성_평균': [abusive[f].mean() for f in key_features],
        '차이': [abusive[f].mean() - normal[f].mean() for f in key_features]
    })
    
    summary.to_csv('../output/feature_comparison.csv', index=False, encoding='utf-8-sig')
    print("✅ 비교 결과 저장: output/feature_comparison.csv")
    print("\n비교 결과:")
    print(summary.round(3))

overall_stats = df.describe().T
overall_stats.to_csv('../output/overall_statistics.csv', encoding='utf-8-sig')
print("\n✅ 전체 통계 저장: output/overall_statistics.csv")

분석 결과 저장
✅ 비교 결과 저장: output/feature_comparison.csv

비교 결과:
                    Feature   정상_평균   악성_평균     차이
0            review_density  20.506  19.918 -0.587
1          question_density  12.918  15.326  2.408
2                avg_rating   4.667   4.790  0.123
3  negative_sentiment_ratio   0.069   0.157  0.087
4      rating_sentiment_gap   0.075   0.142  0.067
5     question_review_ratio   0.625   0.805  0.181

✅ 전체 통계 저장: output/overall_statistics.csv


In [11]:
# HTML로 저장
fig.write_html('../output/analyze_results.html')

## 🎉 완료!

### 생성된 그래프:
- ✅ 라벨 분포 (파이 차트 + 막대 그래프)
- ✅ 조건 충족 개수 분포
- ✅ Feature 비교 Box Plot (6개)
- ✅ 상관관계 히트맵
- ✅ 산점도 (부정 감성 vs 평점 괴리)
- ✅ 주요 판매자 테이블 (상위/하위)

### 인터랙티브 기능:
- 🖱️ 마우스 호버 → 상세 정보
- 🔍 클릭 드래그 → 확대/축소
- 👆 범례 클릭 → 데이터 숨기기/보이기
- 💾 우측 상단 카메라 아이콘 → PNG 다운로드

### 저장된 파일:
- output/feature_comparison.csv
- output/overall_statistics.csv
- output/analyze_results.html